<a href="https://colab.research.google.com/github/tbeucler/2024_TROPICANA_ML/blob/main/Tropicana_ML_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> 🌴 Tropicana 2024 🌴 <br> ML Regression for Short-Range Tropical Cyclone Forecasting 🌀

*Co-authors (in alphabetical order): Tom Beucler, Milton Gomez (UNIL), Marie McGraw (CIRA)*

<center>
<img src="https://drive.google.com/uc?export=view&id=1WG0jBktEJFAfOXpxRAFXtQNy0GFtsHiR" width=75% title="Image of Dorian (2019), image from NASA"></img>
</center>

Hello and welcome to this tutorial, in which you will be learning how to design linear models and random forests to forecast tropical cyclone maximum wind speed and minimum sea level pressure.  

In order to get everything working we need to get access to some data.

Please begin by opening this <a href="https://drive.google.com/drive/folders/1imMvOdZrfKPvYeO4M6s0UlMPYkbEkCj-" target=_blank> google drive link </a> (it should open in a new tab). You'll need to add a shortcut to it on your own google drive account.

<center>
<img src="https://drive.google.com/uc?export=view&id=10Y3jhNAvmBlBu7fu-b-3-oY_9uD2BY4z" width=75% title="Find the add shortcut command"></img>
<img src="https://drive.google.com/uc?export=view&id=1pTE5ZB9tCTLvFOC_LM5aQuIYcTUnH4WA" width=50% title="Add shortcut to home in drive"></img>
</center>

You should now have access to a hosted copy of IBTrACS, a set of netCDFs based on the SHIPS developmental dataset, and training materials kindly provided by <a href="https://marie-mcgraw.github.io/">Marie McGraw</a>. We'll be loading these files into our workspace in a cell below 😄

## Prerequisites
Let's go ahead and do some prep work for the environment by running the cell below.

In [ ]:
#@markdown  Run this cell to pip install some packages we'll be needing and do some additional setup. Double click it if you want to check out the source, and double click this text again to hide it :)
#Note that %%capture allows us to supress cell output
# %%capture
!pip install tcbenchmark==0.0.3rc12
!pip install cartopy
!pip install memory_profiler
!pip install metpy
!pip install dask-ml

import numpy as np
import pandas as pd
import seaborn as sns
import os
import xarray as xr

# To make this notebook's output stable across runs
rnd_seed = 42
rnd_gen = np.random.default_rng(rnd_seed)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Data Preparation

Let's start out by accessing our Google Drive in the notebook. You'll get a popup asking you to authorize the access. It looks like this:

<center>
<img src="https://drive.google.com/uc?export=view&id=1YlGmAYLjcGGdJbRgXO_uw5U2BIekxggT" width=55% title="Authorize access to Google Drive command"></img>
</center>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') # This will mount your entire drive! Unfortunately, you can't mount just a folder.

In [ ]:
# Assuming you, like us, made the shortcut in your base directory in Google Drive, you should be able to list the folders by using the following code:
print(os.listdir('/content/drive/MyDrive/Tropicana_ML_data'))
# ['SHIPS_netcdfs', 'Marie_McGraw', 'IBTrACS'] should be printed to the cell output.

# And with the follow bash command, we can look at some files in the SHIPS_netcdfs folder.
!ls /content/drive/MyDrive/Tropicana_ML_data/SHIPS_netcdfs/ -lah | grep 052019

# And this will be the path to the folder where the IBTrACS .csv is located
!echo /content/drive/MyDrive/Tropicana_ML_data/IBTrACS/

Now that we have access to google drive, we have the data that we want to work with. Let's take a moment to load a sample and see what kind of information we have.

In [ ]:
""" --------------------------------------------------------------------------
We already imported xarray, numpy, pandas, and matplotlib in the setup cell, so
we can just go ahead and open up the file we're interested in. For this example
we've chosen Dorian (2019) and Hagibis (2019)
""";
dorian_data = xr.open_dataset('/content/drive/MyDrive/Tropicana_ML_data/SHIPS_netcdfs/AL052019.nc')
hagibis_data = xr.open_dataset('/content/drive/MyDrive/Tropicana_ML_data/SHIPS_netcdfs/WP202019.nc')

In [ ]:
# Let's get an idea of what data variables we have for these storms:
print(f'We have {len(dorian_data.data_vars)} Dorian Datavars: \n',
      *dorian_data.data_vars,
      f'\n We have {len(hagibis_data.data_vars)} Hagibis Datavars: \n',
      *hagibis_data.data_vars,
      sep=' ')

# This tracks with what we know: SHIPS has a different set of variables for different basins

However, we're interested in looking at seasons of storms and not just individual storms. We're developing a benchmark dataset + framework for tropical cyclone studies called <a href='https://github.com/msgomez06/TCBench_Alpa'> TCBench</a>, so we'll go ahead and rely on it to look at storm seasons.

In [ ]:
from tcbenchmark.utils import toolbox

In [ ]:
# Please define the seasons you would like to study in this notebook HERE
season_list = [*range(2010,2020),] # The default is to load 10 seasons, from 2010 - 2019, for this exercise.

In [ ]:
tracks = toolbox.get_TC_seasons(
    tracks_path='/content/drive/MyDrive/Tropicana_ML_data/IBTrACS/',
    season_list=season_list,
    datadir_path = '/content/drive/MyDrive/Tropicana_ML_data/')

Using TCBench, each storm is loaded as a <i>track</i> object. The `get_TC_seasons` function will return a dictionary with the seasons as keys, and each item will be a list of track objects for the storms that are recorded in IBTrACS for that season.

In [ ]:
# Let's take a single storm as a sample from the 2019 season
test_storm = rnd_gen.choice(tracks[2019])

In [ ]:
# If it's the first time you ran the code cell above, you should have Desmond stored in test_storm. Let's start by looking at some of its attributes
print(
    test_storm.uid, # By default when loading IBTrACS, TCBench will use the SID as the unique identifier
    test_storm.ALT_ID, # and the ATCF ID, which is used by SHIPS, will be stored in the ALT_ID field.
    test_storm.name, # The name (or lack thereof) is stored in name
    test_storm.wind, # This will return the US agency wind timeseries
    test_storm.pressure, # And this the US agency pressure timeseries
    sep = "\n-------\n"
)

There are some wind and pressure values missing in the tracks for Desmond! This would present a problem if we tried to feed it directly into an algorithm. However, TCBench can help us retrive the data we're interested while taking care of those kinds of issues.

First, because we know we'll be working with SHIPS (and therefore need the ATCF ID), let's verify that the storms we're working with have a valid ATCF ID.

In [ ]:
# Some storms may not have an associated ACTF ID (e.g., unnamed storms that were not tracked by a US agency). Let's check if we have any of those in 2019
for track in tracks[2019]:
    if track.ALT_ID is None: print(track, track.ALT_ID)

We've verified that some of the storms in IBTrACS don't have an associated ACTF ID. Let's go ahead and remove those from our lists, since we won't be able to use those for our study. We'll also limit ourselves to the North Atlantic basin to speed things up.

<small>Side note: The TCTrack object currently doesn't have an easily accessible basin attribute, but if you'd like to filter by basin you could do so with the first two letters of the ATCF ID</small>

In [ ]:
for season, storm_list in tracks.items():
    for storm in storm_list.copy():
        if storm.ALT_ID is None or not ('AL' in storm.ALT_ID): storm_list.remove(storm)

In [ ]:
# If we run this again we shouldn't see anything
for track in tracks[2019]:
    if track.ALT_ID is None: print(track, track.ALT_ID)

Now that we have a relatively clean set of tracks, let's go ahead and define the time series that we will generate for our regression algorithm.

Today, we'll be predicting the 24h intensification of a storm using a series of SHIPS predictors, but we'll limit the information to that which was available at forecasting (i.e., we won't be using any predictors indexed for lead_time>0).

We'll also use 5 timesteps for each predictor.

Finally, we have to come up with a definition for our training set, validation set, and test set. These will allow us, respectively, to fit our algorithm, tune our hyperparameters, and objectively evaluate model performance once we're happy with their performance on the training and validation sets.
We'll use 60% of the seasons for training, 20% of the seasons for validation, and 20% of the seasons for testing.

In [ ]:
# We'll define the number of hours into the future we'll be predicting
delta_target = 24

# and store the minimum and maximum leadtimes (both = 0)
leadtime_min = 0
leadtime_max = 0

# And define the number of steps for each predictor.
timeseries_length = 5 # Values of 4 and lower currently cause a crash

# Let's define the splits dictionary
splits = {
    'train': 0.6,
    'validation': 0.2,
    'test': 0.2
}

# And finally, the location for the data folder
timeseries_dir = '/content/drive/MyDrive/Tropicana_ML_data/SHIPS_netcdfs'

In [ ]:
# Note that by default TCBench will serve the following variables for the time series
from tcbenchmark.utils.constants import default_ships_vars
print(default_ships_vars)
# If you want to pass a different set of predictors from the developmental dataset, simply pass the list as the 'variables' argument when calling get_timeseries_sets below.

In [ ]:
# Call the function get_timeseries_sets from the toolbox module
data_dictionary = toolbox.get_timeseries_sets(
    splits=splits,                     # splits: Dictionary specifying the split ratios for train, validation, and test sets.
    season_dict=tracks,                # season_dict: Dictionary containing the track data for each season.
    leadtime_min=leadtime_min,         # leadtime_min: Minimum lead time for the time series data (e.g., -12 hours).
    leadtime_max=leadtime_max,         # leadtime_max: Maximum lead time for the time series data (e.g., 0 hours).
    timeseries_length=timeseries_length, # timeseries_length: Length of the time series data (number of timesteps).
    delta_target=delta_target,         # delta_target: Lead time in hours for the target intensity (e.g., 24 hours).
    timeseries_dir=timeseries_dir,     # timeseries_dir: Directory path where the time series data files are stored.
)

## Data Analysis

In [ ]:
# Iterate and print the structure with data shapes
for main_key in data_dictionary.keys():
    print(f'\n  {main_key}')
    for sub_key in data_dictionary[main_key].keys():
        print(f'  {sub_key}')
        data_item = data_dictionary[main_key][sub_key]
        # Using the fact that data_item is a list or array-like structure
        if hasattr(data_item, 'shape'):  # For numpy arrays, pandas DataFrames, etc.
            print(f'    Shape: {data_item.shape}')
        else:
            print(f'    Value: {data_item}')

In [ ]:
# Extract the training data from the data dictionary
train_data = data_dictionary['train']

# Calculate mean and standard deviation for each training variable
data_means = train_data['data'].mean(axis=(0, 1))  # Mean of all data variables across the first two dimensions
data_stds = train_data['data'].std(axis=(0, 1))  # Standard deviation of all data variables across the first two dimensions
intensity_deltas_means = train_data['intensity_deltas'].mean(axis=0)  # Mean of intensity deltas
intensity_deltas_stds = train_data['intensity_deltas'].std(axis=0)  # Standard deviation of intensity deltas
base_intensities_means = train_data['base_intensities'].mean(axis=0)  # Mean of base intensities
base_intensities_stds = train_data['base_intensities'].std(axis=0)  # Standard deviation of base intensities

# Create a combined list of all variable names
all_vars = default_ships_vars + ['Delta Intensity', 'Delta MSLP', 'Base Intensity', 'Base MSLP']

# Concatenate all means and standard deviations into single arrays
all_means = np.concatenate([data_means, intensity_deltas_means, base_intensities_means])
all_stds = np.concatenate([data_stds, intensity_deltas_stds, base_intensities_stds])

In [ ]:
# Plot PDFs (Probability Density Functions) with statistics for each variable
plt.figure(figsize=(20, 15))  # Set the figure size
num_vars = len(all_vars)  # Total number of variables
for i in range(num_vars):
    plt.subplot(6, 4, i + 1)  # Create a subplot for each variable in a 6x4 grid
    if i < 18:  # First 18 variables (ship data)
        var_data = train_data['data'][:, :, i].flatten()  # Flatten the 3D array to 1D for histogram
    elif i < 20:  # Next 2 variables (intensity deltas)
        var_data = train_data['intensity_deltas'][:, i - 18]  # Select the intensity delta variables
    else:  # Last 2 variables (base intensities)
        var_data = train_data['base_intensities'][:, i - 20]  # Select the base intensity variables

    plt.hist(var_data, bins=30, density=True, alpha=0.6, color='k')  # Plot the histogram with density
    plt.title(f"{all_vars[i]}\nMean: {all_means[i]:.2f}, Std: {all_stds[i]:.2f}")  # Title with variable name, mean, and std
    plt.xlabel(all_vars[i])  # X-axis label with variable name
    plt.ylabel('Density')  # Y-axis label as 'Density'

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()  # Display the plot

In [ ]:
# Extract the first intensity_deltas variable (Delta Intensity)
delta_intensity = train_data['intensity_deltas'][:, 0]

# Calculate correlations between each variable at each timestep and the first intensity_deltas variable
timesteps = train_data['data'].shape[1]
correlations = np.zeros((18, timesteps))

for var in range(18):
    for t in range(timesteps):
        var_data = train_data['data'][:, t, var]
        correlations[var, t] = np.corrcoef(var_data, delta_intensity)[0, 1]

# Create time labels assuming each timestep represents a 6-hour interval
time_labels = np.flip(['t_IC', 't_IC - 6hr', 't_IC - 12hr', 't_IC - 18hr', 't_IC - 24hr'])
# where t_IC is the initial conditions time
# We're trying to predict the intensity at t_IC+24hr from t_IC

# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, annot=True, cmap='coolwarm', xticklabels=time_labels, yticklabels=default_ships_vars, center=0)

plt.xlabel('Time')
plt.ylabel('Variables')
plt.title('Pearson Correlation of Variables with Delta Intensity')
plt.show()

In [ ]:
# Extract the second intensity_deltas variable (Delta MSLP)
delta_mslp = train_data['intensity_deltas'][:, 1]

# Calculate correlations between each variable at each timestep and Delta MSLP
timesteps = train_data['data'].shape[1]
correlations = np.zeros((18, timesteps))

for var in range(18):
    for t in range(timesteps):
        var_data = train_data['data'][:, t, var]
        correlations[var, t] = np.corrcoef(var_data, delta_mslp)[0, 1]

# Create time labels assuming each timestep represents a 6-hour interval
time_labels = np.flip(['t_IC', 't_IC - 6hr', 't_IC - 12hr', 't_IC - 18hr', 't_IC - 24hr'])
# where t_IC is the initial conditions time
# We're trying to predict the intensity at t_IC+24hr from t_IC

# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, annot=True, cmap='coolwarm',
            xticklabels=time_labels, yticklabels=default_ships_vars,
            center=0)

plt.xlabel('Time')
plt.ylabel('Variables')
plt.title('Pearson Correlation of Variables with Delta Mean Sea Level Pressure')
plt.show()

## Data Pre-processing

### For a 24-hour lead time prediction of Delta (Intensity, MSLP) without memory

In [ ]:
def normalize_data(X, mean, std):
    """
    Normalize the data using the provided mean and standard deviation.

    Parameters:
    - X: input data to be normalized
    - mean: mean for normalization
    - std: standard deviation for normalization

    Returns:
    - X_normalized: normalized data
    """
    return (X - mean) / std

def denormalize_data(X_normalized, mean, std):
    """
    Denormalize the data using the provided mean and standard deviation.

    Parameters:
    - X_normalized: normalized data to be denormalized
    - mean: mean for denormalization
    - std: standard deviation for denormalization

    Returns:
    - X: denormalized data
    """
    return X_normalized * std + mean

In [ ]:
# Extract the training, validation, and test data from the data dictionary
train_data = data_dictionary['train']
val_data = data_dictionary['validation']
test_data = data_dictionary['test']

# Extract X and Y for training
X_train = train_data['data'][:, 0, 1:]  # Use only the first timestep (column 0) for all samples, and eliminate the first variable (DELV)
Y_train = train_data['intensity_deltas']  # Use both Delta Intensity and Delta MSLP

# Normalize X and Y for training
X_train_normalized = normalize_data(X_train, data_means[1:], data_stds[1:])
Y_train_normalized = normalize_data(Y_train, intensity_deltas_means, intensity_deltas_stds)

# Extract and normalize X and Y for validation
X_val = val_data['data'][:, 0, 1:]
Y_val = val_data['intensity_deltas']
X_val_normalized = normalize_data(X_val, data_means[1:], data_stds[1:])
Y_val_normalized = normalize_data(Y_val, intensity_deltas_means, intensity_deltas_stds)

# Extract and normalize X and Y for test
X_test = test_data['data'][:, 0, 1:]
Y_test = test_data['intensity_deltas']
X_test_normalized = normalize_data(X_test, data_means[1:], data_stds[1:])
Y_test_normalized = normalize_data(Y_test, intensity_deltas_means, intensity_deltas_stds)

# Create metadata
X_metadata = default_ships_vars[1:]  # Exclude the first variable (DELV)
Y_metadata = ['Delta Intensity', 'Delta MSLP']

### For a 24-hour lead time prediction of Delta (Intensity, MSLP) with memory

### For an auto-regressive model

Coming soon. You may ask [Chia-Ying Lee](https://journals.ametsoc.org/view/journals/clim/29/21/jcli-d-15-0909.1.xml?tab_body=pdf) for advice 😃

## Machine Learning Model Hierarchy

Here, we develop a model hierarchy targeting the 24-hour intensification rate. These models will serve as baselines for comparison. For each model, we will keep track of the error statistics on the training, validation, and test sets, as well as the number of parameters and the nature of the model throughout the process.

In [ ]:
# Initialize an empty dictionary to accumulate all model scores
model_scores = {}

In [ ]:
# @markdown Run to install ML libraries
%%capture
!pip install xgboost
!pip install adjustText

In [ ]:
# @markdown Run to import ML libraries

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.base import clone, BaseEstimator, RegressorMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold, PredefinedSplit, RandomizedSearchCV

import xgboost as xgb

from adjustText import adjust_text

### Scoring functions

In [ ]:
def score_model(model, X_train, Y_train, X_val=None, Y_val=None, X_test=None, Y_test=None, model_name="model", num_params=None):
    """
    Function to score the model and accumulate the scores in a dictionary for training, validation, and test sets.

    Parameters:
    - model: the trained regression model
    - X_train: the input features for training (normalized)
    - Y_train: the target values for training (normalized)
    - X_val: the input features for validation (optional, normalized)
    - Y_val: the target values for validation (optional, normalized)
    - X_test: the input features for test (optional, normalized)
    - Y_test: the target values for test (optional, normalized)
    - model_name: a name for the model (default is "model")
    - num_params: the number of trainable parameters (optional)

    Accumulates:
    - model_scores: a dictionary containing RMSE, MAE, R² per variable, and number of parameters for each model on training, validation, and test sets
    """
    global model_scores

    def calculate_scores(X, Y):
        """
        Helper function to calculate RMSE, MAE, and R² scores.
        """
        Y_pred = model.predict(X)
        Y_denorm = denormalize_data(Y, intensity_deltas_means, intensity_deltas_stds)
        Y_pred_denorm = denormalize_data(Y_pred, intensity_deltas_means, intensity_deltas_stds)

        rmse_knots = np.sqrt(mean_squared_error(Y_denorm[:, 0], Y_pred_denorm[:, 0]))
        mae_knots = mean_absolute_error(Y_denorm[:, 0], Y_pred_denorm[:, 0])
        rmse_hPa = np.sqrt(mean_squared_error(Y_denorm[:, 1], Y_pred_denorm[:, 1]))
        mae_hPa = mean_absolute_error(Y_denorm[:, 1], Y_pred_denorm[:, 1])
        r2_wind = r2_score(Y[:, 0], Y_pred[:, 0])
        r2_pressure = r2_score(Y[:, 1], Y_pred[:, 1])
        return rmse_knots, mae_knots, rmse_hPa, mae_hPa, r2_wind, r2_pressure

    # Initialize score dictionary for the current model
    score_dict = {}

    # Calculate and store scores for the training set
    train_rmse_knots, train_mae_knots, train_rmse_hPa, train_mae_hPa, train_r2_wind, train_r2_pressure = calculate_scores(X_train, Y_train)
    score_dict['train'] = {
        "RMSE (knots)": train_rmse_knots,
        "MAE (knots)": train_mae_knots,
        "RMSE (hPa)": train_rmse_hPa,
        "MAE (hPa)": train_mae_hPa,
        "R² wind": train_r2_wind,
        "R² pressure": train_r2_pressure
    }

    # Calculate and store scores for the validation set (if provided)
    if X_val is not None and Y_val is not None:
        val_rmse_knots, val_mae_knots, val_rmse_hPa, val_mae_hPa, val_r2_wind, val_r2_pressure = calculate_scores(X_val, Y_val)
        score_dict['validation'] = {
            "RMSE (knots)": val_rmse_knots,
            "MAE (knots)": val_mae_knots,
            "RMSE (hPa)": val_rmse_hPa,
            "MAE (hPa)": val_mae_hPa,
            "R² wind": val_r2_wind,
            "R² pressure": val_r2_pressure
        }

    # Calculate and store scores for the test set (if provided)
    if X_test is not None and Y_test is not None:
        test_rmse_knots, test_mae_knots, test_rmse_hPa, test_mae_hPa, test_r2_wind, test_r2_pressure = calculate_scores(X_test, Y_test)
        score_dict['test'] = {
            "RMSE (knots)": test_rmse_knots,
            "MAE (knots)": test_mae_knots,
            "RMSE (hPa)": test_rmse_hPa,
            "MAE (hPa)": test_mae_hPa,
            "R² wind": test_r2_wind,
            "R² pressure": test_r2_pressure
        }

    # Get the number of trainable parameters
    if num_params is None:
        if isinstance(model, LinearRegression):
            num_params = model.coef_.size + model.intercept_.size
        elif isinstance(model, RandomForestRegressor):
            num_params = sum(estimator.tree_.node_count for estimator in model.estimators_)
        elif isinstance(model, xgb.XGBRegressor):
            # Careful: This is a heuristic...
            n_estimators = model.get_params()['n_estimators']
            max_depth = model.get_params()['max_depth']
            # Estimate the number of nodes per tree
            nodes_per_tree = 2 ** max_depth - 1
            # Total number of nodes in the model
            num_params = n_estimators * nodes_per_tree
        else:
            num_params = "N/A"  # For other models, we might not have a straightforward way to calculate parameters


    # Add the number of parameters to the score dictionary
    score_dict['num_params'] = num_params

    # Accumulate the scores in the global model_scores dictionary
    model_scores[model_name] = score_dict

# Print the accumulated model scores in a readable way
def print_model_scores(model_scores):
    for model_name, scores in model_scores.items():
        print(f"Model: {model_name}")
        for dataset, metrics in scores.items():
            if dataset != 'num_params':
                print(f"  {dataset.capitalize()}:")
                for metric, value in metrics.items():
                    print(f"    {metric}: {value:.4f}")
            else:
                print(f"  Number of Parameters: {metrics}")
        print()

### Multiple Linear Regression
<center>
<img src="https://drive.google.com/uc?export=view&id=1JD1qaj7J077udgMwCQdkvNPPqlNtUzNd" width=15% title="Epicycloid image by the Mathematics of Waves and Materials Group @ the University of Manchester"></img> <img src="https://drive.google.com/uc?export=view&id=1rjXW5I8mxoAFVd7pmtySHBGoDUfpuWIo" width=15% title="Epicycloid image by the Mathematics of Waves and Materials Group @ the University of Manchester"></img> <img src="https://drive.google.com/uc?export=view&id=1JD1qaj7J077udgMwCQdkvNPPqlNtUzNd" width=15% title="Epicycloid image by the Mathematics of Waves and Materials Group @ the University of Manchester"></img> <img src="https://drive.google.com/uc?export=view&id=1rjXW5I8mxoAFVd7pmtySHBGoDUfpuWIo" width=15% title="Epicycloid image by the Mathematics of Waves and Materials Group @ the University of Manchester"></img> <img src="https://drive.google.com/uc?export=view&id=1JD1qaj7J077udgMwCQdkvNPPqlNtUzNd" width=15% title="Epicycloid image by the Mathematics of Waves and Materials Group @ the University of Manchester"></img>

</center>

In [ ]:
# Fit the linear regression model
model = LinearRegression()
model.fit(X_train_normalized, Y_train_normalized)

# Score the model on training, validation, and test sets
score_model(model, X_train_normalized, Y_train_normalized, X_val_normalized, Y_val_normalized, X_test_normalized, Y_test_normalized, model_name="Linear Regression")

# Print the accumulated scores dictionary
print_model_scores(model_scores)

In [ ]:
# For comparison, score the persistence model
class Persistence(BaseEstimator, RegressorMixin):
    """
    A persistence model that predicts zero for any input in the denormalized space.
    """
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def fit(self, X, y):
        return self  # Nothing to do here as predictions are constant.

    def predict(self, X):
        # Calculate the normalized value that corresponds to zero in the denormalized space
        return -self.mean / self.std * np.ones((X.shape[0], len(self.mean)))

In [ ]:
# Instantiate the persistence model using the means and stds for Y
persistence_model = Persistence(mean=intensity_deltas_means, std=intensity_deltas_stds)

# Fit the persistence model (though it does nothing)
persistence_model.fit(X_train_normalized, Y_train_normalized)

# Score the persistence model on training, validation, and test sets
score_model(persistence_model, X_train_normalized, Y_train_normalized,
            X_val_normalized, Y_val_normalized, X_test_normalized,
            Y_test_normalized, model_name="Persistence", num_params=1)
# num_params = 1 to avoid singularity

# Print the accumulated scores dictionary
print_model_scores(model_scores)

This is not great and it even looks like we're overfitting a bit! Can we do better by only selecting the best features?

### Forward Sequential Feature Selection

Forward feature sequential selection is a greedy algorithm that adds one input at a time based on a cross-validated score until the addition of further inputs does not improve the model's performance.

<center>
<img src="https://drive.google.com/uc?export=view&id=1tGCD3ALPAp_a-Fjgrnx9dIgaxHruEEhC" width=75% title="Feature Selection, src: Adobe Stock"></img>
</center>

In [ ]:
def forward_feature_selection(X_train, Y_train, X_val, Y_val, feature_names, objective, model_class=LinearRegression):
    """
    Perform forward sequential feature selection based on a custom objective.

    Parameters:
    - X_train: the input features for training (normalized)
    - Y_train: the target values for training (normalized)
    - X_val: the input features for validation (normalized)
    - Y_val: the target values for validation (normalized)
    - feature_names: names of the features
    - objective: function to optimize during feature selection, should return a score
    - model_class: the regression model class to be used (default is LinearRegression)

    Returns:
    - best_features: indices of the best selected features
    - best_model: the best trained model
    """
    n_features = X_train.shape[1]  # Total number of features
    selected_features = []  # List to hold the indices of selected features
    remaining_features = list(range(n_features))  # List of indices of features not yet selected
    best_score = float('-inf')  # Initialize the best score to negative infinity
    best_model = None  # Placeholder for the best model
    best_features = []  # Placeholder for the indices of the best features

    while remaining_features:
        current_best_score = float('-inf')  # Best score for the current iteration
        current_best_feature = None  # Best feature for the current iteration
        current_best_model = None  # Best model for the current iteration

        # Evaluate each remaining feature to find the best one to add
        for feature in remaining_features:
            features_to_try = selected_features + [feature]  # Current set of features to try
            model = model_class()  # Instantiate the model
            model.fit(X_train[:, features_to_try], Y_train)  # Fit the model on the training data
            score = objective(model, X_val[:, features_to_try], Y_val)  # Evaluate the model using the objective function

            # Update the current best feature, score, and model if the score improves
            if score > current_best_score:
                current_best_score = score
                current_best_feature = feature
                current_best_model = clone(model)

        # Add the best feature of the current iteration to the selected features
        selected_features.append(current_best_feature)
        remaining_features.remove(current_best_feature)

        # Update the overall best score, model, and features if the current score is better
        if current_best_score > best_score:
            best_score = current_best_score
            best_model = current_best_model
            best_features = selected_features.copy()

        # Fit the best model with the current best features
        best_model = model_class()
        best_model.fit(X_train[:, selected_features], Y_train)
        best_score = current_best_score

    return best_features, best_model

We want to see whether the best linear model to predict wind is the same as the best model to predict pressure.

In [ ]:
# Define custom objective functions
def r2_wind_objective(model, X, Y):
    Y_pred = model.predict(X)
    return r2_score(Y[:, 0], Y_pred[:, 0])

def r2_pressure_objective(model, X, Y):
    Y_pred = model.predict(X)
    return r2_score(Y[:, 1], Y_pred[:, 1])

In [ ]:
# Perform forward sequential feature selection for wind and pressure
feature_names = default_ships_vars[1:]  # Exclude the first variable (DELV)

best_features_wind, best_model_wind = forward_feature_selection(
    X_train_normalized, Y_train_normalized, X_val_normalized, Y_val_normalized,
    feature_names, r2_wind_objective, model_class=LinearRegression
)

best_features_pressure, best_model_pressure = forward_feature_selection(
    X_train_normalized, Y_train_normalized, X_val_normalized, Y_val_normalized,
    feature_names, r2_pressure_objective, model_class=LinearRegression
)

# Print the selected features for wind and pressure
print("Selected feature indices for wind:", best_features_wind)
print("Selected feature names for wind:", [feature_names[i] for i in best_features_wind])
print("Selected feature indices for pressure:", best_features_pressure)
print("Selected feature names for pressure:", [feature_names[i] for i in best_features_pressure])

In [ ]:
# Refit the best models with the selected features
best_model_wind.fit(X_train_normalized[:, best_features_wind], Y_train_normalized)
best_model_pressure.fit(X_train_normalized[:, best_features_pressure], Y_train_normalized)

# Calculate and print scores for the best models, including the test set
score_model(best_model_wind,
            X_train_normalized[:, best_features_wind], Y_train_normalized,
            X_val_normalized[:, best_features_wind], Y_val_normalized,
            X_test_normalized[:, best_features_wind], Y_test_normalized,
            model_name="SFS Linear Regression (Wind)")

score_model(best_model_pressure,
            X_train_normalized[:, best_features_pressure], Y_train_normalized,
            X_val_normalized[:, best_features_pressure], Y_val_normalized,
            X_test_normalized[:, best_features_pressure], Y_test_normalized,
            model_name="SFS Linear Regression (Pressure)")

print_model_scores(model_scores)

This works! We avoided overfitting our linear regression model by strategically selecting our features 😀

Now let's see if we can do the same with a random forest 🌲

### Random Forest

<center>
<img src="https://drive.google.com/uc?export=view&id=1NSvebXiF9yW_PRoFKMfFRn2eAKA7hMEa" width=75% title="Bamboo Forest, src: Adobe Stock"></img>
</center>

#### First attempt

In [ ]:
# Initialize a RandomForestRegressor with default parameters
naive_rf_model = RandomForestRegressor()

# Fit the model using the normalized training set
naive_rf_model.fit(X_train_normalized, Y_train_normalized)

In [ ]:
# Score the naive Random Forest model
score_model(naive_rf_model,
            X_train_normalized, Y_train_normalized,
            X_val_normalized, Y_val_normalized,
            X_test_normalized, Y_test_normalized,
            model_name="Naive Random Forest")

print("Accumulated Model Scores:")
print_model_scores(model_scores)

#### Hyperparameter sweep

To set up a hyperparameter sweep and data pipeline for a Random Forest model, we will follow these steps:

1. Data Preparation: Use the normalized training set (X_train_normalized, Y_train_normalized).

2. Model and Hyperparameters:

* Define the Random Forest model and set its parameters: max_depth, n_estimators, max_features, min_samples_leaf.
* Specify the scoring function (r2 for R² score).
* Use the 'squared_error' criterion for the Random Forest.

3. Cross-Validation: Set up cross-validation parameters using KFold.

4. Model Fitting: Fit the Random Forest model to X_train_normalized and Y_train_normalized.

Random Forest Model Definitions

* criterion: Use 'squared_error' for mean squared error.
* max_features: For regression, typically use N_features, but other values like sqrt(N_features) or N_features/3 are also common. Options: [5, 6, 15, 17].
* n_estimators: Number of decision trees. Options: [100, 250].
* min_samples_leaf: Minimum number of samples required to create a leaf node. Options: [2, 4, 8].
* max_depth: Maximum depth of each decision tree. Options: [5, 6, 8].

Cross-Validation Definitions

* k_folds: Number of folds for cross-validation. We will use 10 folds.

In [ ]:
def create_gridsearch_RF_regressor(score, max_depth, n_estimators, max_features,
                                   min_samples_leaf, cv, scoring_metric='r2'):
    """
    Create a GridSearchCV object for a RandomForestRegressor.

    Parameters:
    - score: list of scoring functions to use (e.g., ['squared_error'])
    - max_depth: list of max_depth values to try
    - n_estimators: list of n_estimators values to try
    - max_features: list of max_features values to try
    - min_samples_leaf: list of min_samples_leaf values to try
    - cv: cross-validation splitting strategy (e.g., PredefinedSplit object)
    - scoring_metric: scoring metric to use (default is 'r2')

    Returns:
    - rfgs: GridSearchCV object
    """
    rfmodel = RandomForestRegressor()
    params = {
        'max_depth': max_depth,
        'criterion': score,
        'max_features': max_features,
        'n_estimators': n_estimators,
        'min_samples_leaf': min_samples_leaf
    }
    rfgs = GridSearchCV(rfmodel, param_grid=params, cv=cv, scoring=scoring_metric)
    return rfgs

In [ ]:
# Hyperparameter options
# This may be a bit long depending on how many HPs you decide to try out
# Proceed with caution
k_folds = 10
max_depth = [5, 6, 8]
score = ['squared_error']
max_features = [5, 6, 15, 17]
n_estimators = [100, 250]
min_samples_leaf = [2, 4, 8]

In [ ]:
# To perform validation on the pre-defined validation set,
# We create a PredefinedSplit object
# `-1` indicates the training set, and any other value indicates the validation set.
trval_fold = np.concatenate([
    np.full(X_train_normalized.shape[0], -1),
    np.zeros(X_val_normalized.shape[0])
])

# Combine the training and validation sets
X_combined = np.concatenate([X_train_normalized, X_val_normalized])
Y_combined = np.concatenate([Y_train_normalized, Y_val_normalized])

ps = PredefinedSplit(test_fold=trval_fold)

# Create GridSearchCV object
rfgs = create_gridsearch_RF_regressor(score, max_depth, n_estimators,
                                      max_features, min_samples_leaf, ps)

# Fit the model using the combined training and validation sets
rfgs.fit(X_combined, Y_combined)

In [ ]:
# Get the best model and parameters
best_model = rfgs.best_estimator_
best_params = rfgs.best_params_

print("Best Model Parameters:", best_params)

If you don't have time to optimize hyperparameters, we found the following best model parameters:

Best Model Parameters: `{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 8, 'n_estimators': 100}`

In [ ]:
# Calculate and print scores for the best model, including the test set
score_model(best_model,
            X_train_normalized, Y_train_normalized,
            X_val_normalized, Y_val_normalized,
            X_test_normalized, Y_test_normalized,
            model_name="Best Random Forest")

In [ ]:
print_model_scores(model_scores)

The naive random forest is clearly overfitting! Let's cut down the number of features using explainable artificial intelligence methods 🧠

#### Gini-impurity based feature selection

**Gini Impurity**:
- Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the set.
- It is used by the CART (Classification and Regression Tree) algorithm for classification problems.
- The formula for the Gini impurity can be found on [Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning#:~:text=Gini%20impurity%20measures%20how%20often,into%20a%20single%20target%20category.)

**Feature Importance Using Gini Impurity**:
- Feature importance in the context of decision trees and random forests refers to the contribution of each feature to the prediction accuracy of the model.
- Random forests calculate feature importance by looking at the decrease in Gini impurity caused by a feature.
- For each feature, the random forest algorithm computes the total reduction in Gini impurity brought by that feature across all trees in the forest. The average of these reductions is used as the importance score for the feature.

In [ ]:
# Calculate Feature Importance for the best RF model
feature_importances = best_model.feature_importances_

In [ ]:
# Plot the feature importances and label them using X_metadata
# Using horizontal bars, from top (the most important feature)
# to bottom (the least important feature)
# Sort features by importance
sorted_indices = np.argsort(feature_importances)[::-1]
sorted_features = [feature_names[i] for i in sorted_indices]
sorted_importances = feature_importances[sorted_indices]

# Plot the feature importances
plt.figure(figsize=(10, 8))
plt.barh(range(len(sorted_importances)), sorted_importances, align='center')
plt.yticks(range(len(sorted_importances)), sorted_features)
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.xlabel('Feature Importance')
plt.title('Feature Importances Using Gini Impurity')
plt.show()

Clearly the top 4 features stand out! Let's retrain the model accordingly.

In [ ]:
# Select Top 4 Features
top_4_features_indices = np.argsort(feature_importances)[-4:]

# Filter training and validation sets to keep only the top 4 features
X_train_top_4 = X_train_normalized[:, top_4_features_indices]
X_val_top_4 = X_val_normalized[:, top_4_features_indices]

trval_fold = np.concatenate([
    np.full(X_train_top_4.shape[0], -1),
    np.zeros(X_val_top_4.shape[0])
])

# Combine the training and validation sets
X_combined = np.concatenate([X_train_top_4, X_val_top_4])
Y_combined = np.concatenate([Y_train_normalized, Y_val_normalized])

ps = PredefinedSplit(test_fold=trval_fold)

# Create GridSearchCV object
rfgs_top_4 = create_gridsearch_RF_regressor(score, max_depth, n_estimators,
                                      max_features, min_samples_leaf, ps)

In [ ]:
# Fit the model using the training set with top 4 features
rfgs_top_4.fit(X_combined, Y_combined)

# Get the best model and parameters
best_model_top_4 = rfgs_top_4.best_estimator_
best_params_top_4 = rfgs_top_4.best_params_

print("Best Model Parameters for Top 4 Features:", best_params_top_4)

In [ ]:
# Score the model using the validation set with top 4 features
score_model(best_model_top_4,
            X_train_top_4, Y_train_normalized,
            X_val_top_4, Y_val_normalized,
            X_test_normalized[:, top_4_features_indices], Y_test_normalized,
            model_name="RF Top 4 features")

In [ ]:
print_model_scores(model_scores)

This is still disappointing. Can XGBoost do better?

### XGBoost

**XGBoost Advantages:**

1. **Boosting Technique**: XGBoost uses boosting to build models that focus on correcting errors from previous ones, resulting in a strong, accurate ensemble model.

2. **Regularization**: It includes L1 and L2 regularization to prevent overfitting and improve generalization, especially with complex datasets.

3. **Handling Missing Values**: XGBoost can handle missing values during training, making it suitable for real-world datasets.

4. **Parallel Processing**: XGBoost uses parallel processing, making it faster and more efficient, especially with large datasets.

**Advantages of Random Hyperparameter Search (over e.g., grid search):**

1. **Efficiency**: Random search explores a broader range of hyperparameters by sampling randomly, often finding better configurations with fewer trials.

2. **Scalability**: It can be easily parallelized to test multiple hyperparameter combinations at once, making it faster than grid search.

3. **Continuous Hyperparameters**: Random search handles continuous hyperparameters more effectively by sampling from a continuous distribution, increasing the chance of finding optimal values.

In [ ]:
def create_randomsearch_XGB_regressor(learning_rate, max_depth, n_estimators,
                                      colsample_bytree, min_child_weight, cv,
                                      scoring_metric='r2', n_iter=100):
    """
    Create a RandomizedSearchCV object for an XGBRegressor.

    Parameters:
    - learning_rate: list of learning rate values to try
    - max_depth: list of max_depth values to try
    - n_estimators: list of n_estimators values to try
    - colsample_bytree: list of colsample_bytree values to try
    - min_child_weight: list of min_child_weight values to try
    - cv: cross-validation splitting strategy (e.g., PredefinedSplit object)
    - scoring_metric: scoring metric to use (default is 'r2')
    - n_iter: number of parameter settings that are sampled (default is 100)

    Returns:
    - xgb_rs: RandomizedSearchCV object
    """
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror')
    param_distributions = {
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'n_estimators': n_estimators,
        'colsample_bytree': colsample_bytree,
        'min_child_weight': min_child_weight
    }
    xgb_rs = RandomizedSearchCV(xgb_model, param_distributions=param_distributions,
                                cv=cv, scoring=scoring_metric, n_iter=n_iter, random_state=42)
    return xgb_rs

In [ ]:
# Hyperparameter options for XGBoost

# Learning rate (also known as eta) controls the step size at each iteration while moving toward a minimum of the loss function.
learning_rate = [0.01, 0.05, 0.1, 0.2, 0.3]  # Try different learning rates

# Max depth determines the maximum depth of the trees.
# Deeper trees can model more complex patterns but may overfit.
max_depth = [3, 5, 7, 9, 12]  # Try different maximum depths for the trees

# Number of estimators is the number of trees in the model (ensemble size).
# More trees can improve performance but also increase computation time.
n_estimators = [50, 100, 200, 300]  # Try different numbers of trees

# Colsample_bytree is the subsample ratio of columns when constructing each tree.
# It is used to prevent overfitting by sampling a fraction of features.
colsample_bytree = [0.3, 0.5, 0.7, 0.9, 1.0]  # Try different subsample ratios for columns

# Min child weight is the minimum sum of instance weight (Hessian) needed in a child. It is used to control overfitting.
min_child_weight = [1, 3, 5, 7]  # Try different minimum child weights

# Create a PredefinedSplit object
# `-1` indicates the training set, and `0` indicates the validation set.
test_fold = np.concatenate([
    np.full(X_train_normalized.shape[0], -1),
    np.zeros(X_val_normalized.shape[0])
])

# Combine the training and validation sets
X_combined = np.concatenate([X_train_normalized, X_val_normalized])
Y_combined = np.concatenate([Y_train_normalized, Y_val_normalized])

ps = PredefinedSplit(test_fold=test_fold)

# Create RandomizedSearchCV object for XGBoost
xgb_rs = create_randomsearch_XGB_regressor(learning_rate, max_depth, n_estimators,
                                           colsample_bytree, min_child_weight, ps)

In [ ]:
# Fit the model using the combined training and validation sets
xgb_rs.fit(X_combined, Y_combined)

# Get the best model and parameters
best_xgb_model = xgb_rs.best_estimator_
best_xgb_params = xgb_rs.best_params_

print("Best Model Parameters for XGBoost:", best_xgb_params)

In [ ]:
# Show feature importance
xgb.plot_importance(best_xgb_model)
plt.show()

In [ ]:
# Score the model using the validation set with all features
score_model(best_xgb_model,
            X_train_normalized, Y_train_normalized,
            X_val_normalized, Y_val_normalized,
            X_test_normalized, Y_test_normalized,
            model_name="XGBoost All features")

In [ ]:
print_model_scores(model_scores)

## Now it's your turn: Create your own model 🖌
🎨 Be creative 🎵

You may consider polynomial models, symbolic regression, other types of neural networks, etc.

Given that some variables are clearly more informative, you could use memory and more timesteps to leverage finer temporal information.

Of course, if you use more information, be mindful to regularize your model, e.g., using L1/L2 regularization. You may for instance use an [Elastic Net](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html).

In [ ]:
# Create your model here

In [ ]:
# Don't forget to score it
?score_model

## Place all models on a Pareto front

Pareto fronts allow to easily compare two competing objectives, such as complexity (e.g., the number of trainable parameters) and error

In [ ]:
# Function to determine Pareto optimal points

def is_pareto_efficient_simple(costs):

    """
    Identify the Pareto-efficient points for a given set of costs.

    Parameters:
    costs (np.array): A 2D NumPy array where each row represents an individual point
                      in terms of its cost dimensions. For instance, each row could
                      represent a different configuration's cost as [cost1, cost2].

    Returns:
    np.array: A boolean array where True indicates that the corresponding point in
              'costs' is Pareto efficient.

    Description:
    This function iterates over each point and determines whether it is dominated
    by any other point. A point is considered Pareto efficient if no other point
    exists that is better in all cost dimensions. In this context, "better" is
    defined as being lower for each cost dimension since we are minimizing.

    The function uses a boolean array, `is_efficient`, to keep track of which
    points are currently considered Pareto efficient. Initially, all points are
    assumed to be Pareto efficient.

    For each point:
    1. If the point is still considered potentially Pareto efficient (`is_efficient[i]` is True),
       it checks against all other points that are also still marked as efficient.
    2. It updates the `is_efficient` array by setting it to False for any point
       that is dominated by the current point. A point `j` is dominated by point `i`
       if all its cost dimensions are greater than or equal to those of point `i`
       and at least one dimension is strictly greater.
    3. It ensures the current point's efficiency status remains True regardless of the
       comparison outcome by resetting `is_efficient[i]` to True after the comparison.

    The loop ensures that every point is compared with all others, and after processing,
    the `is_efficient` array flags only those points that are not dominated by any other,
    indicating the Pareto front.
    """

    is_efficient = np.ones(costs.shape[0], dtype=bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            is_efficient[is_efficient] = np.any(costs[is_efficient] < c, axis=1)
            is_efficient[i] = True
    return is_efficient

In [ ]:
# Extracting data into a DataFrame
data = []
for model, scores in model_scores.items():
    test_scores = scores['test']
    num_params = scores['num_params']
    data.append([
        model,
        test_scores["RMSE (knots)"],
        test_scores["RMSE (hPa)"],
        num_params,
        np.log10(num_params)
    ])

df = pd.DataFrame(data, columns=["Model", "Test_RMSE_knots", "Test_RMSE_hPa", "Number of Parameters", "log10_Number of Parameters"])

# Define the cost matrix with log10(Number of Parameters) and Test RMSE for hPa
costs_hPa = df[['log10_Number of Parameters', 'Test_RMSE_hPa']].values

# Find Pareto efficient points for RMSE (hPa)
pareto_efficient_hPa = is_pareto_efficient_simple(costs_hPa)
pareto_models_hPa = df[pareto_efficient_hPa].sort_values(by=['log10_Number of Parameters'])

# Define the cost matrix with log10(Number of Parameters) and Test RMSE for knots
costs_knots = df[['log10_Number of Parameters', 'Test_RMSE_knots']].values

# Find Pareto efficient points for RMSE (knots)
pareto_efficient_knots = is_pareto_efficient_simple(costs_knots)
pareto_models_knots = df[pareto_efficient_knots].sort_values(by=['log10_Number of Parameters'])

# Display the Pareto optimal models
print("Pareto Optimal Models for RMSE (hPa):")
print(pareto_models_hPa)
print("\nPareto Optimal Models for RMSE (knots):")
print(pareto_models_knots)

In [ ]:
# Assuming df, pareto_models_hPa, and pareto_models_knots are defined

# Plot setup
fig, ax = plt.subplots(2, 1, figsize=(11.5, 9.5))

# Plot for RMSE (hPa)
colors = plt.cm.get_cmap('tab10', len(df))
texts = []
for i, row in df.iterrows():
    ax[0].scatter(row['log10_Number of Parameters'],
                  row['Test_RMSE_hPa'], color=colors(i), label=row['Model'])
    texts.append(ax[0].text(row['log10_Number of Parameters'],
                            row['Test_RMSE_hPa']+.01, row['Model'], fontsize=9))

ax[0].set_xlabel('log10(Number of Parameters)')
ax[0].set_ylabel('Test RMSE (hPa)')
ax[0].set_title('Pareto Front for MSLP (hPa)')

pareto_params_hPa = pareto_models_hPa['log10_Number of Parameters'].values
pareto_rmse_hPa = pareto_models_hPa['Test_RMSE_hPa'].values

# Draw the Pareto front for RMSE (hPa)
for i in range(len(pareto_params_hPa)):
    if i > 0:
        ax[0].vlines(x=pareto_params_hPa[i], ymin=pareto_rmse_hPa[i-1],
                     ymax=pareto_rmse_hPa[i], color='k', linewidth=1, zorder=-1)
    ax[0].hlines(y=pareto_rmse_hPa[i],
                 xmin=pareto_params_hPa[i],
                 xmax=pareto_params_hPa[i+1] if i+1 < len(pareto_params_hPa) \
                 else max(df['log10_Number of Parameters']),
                 color='k', linewidth=1, zorder=-1)

# Adjust text positions to avoid overlap
adjust_text(texts, ax=ax[0], only_move={'points':'y', 'texts':'y'},
            arrowprops=dict(arrowstyle='-', color='grey'))

# Plot for RMSE (knots)
texts = []
for i, row in df.iterrows():
    ax[1].scatter(row['log10_Number of Parameters'], row['Test_RMSE_knots'],
                  color=colors(i), label=row['Model'])
    texts.append(ax[1].text(row['log10_Number of Parameters'], row['Test_RMSE_knots']+.01,
                            row['Model'], fontsize=9))

ax[1].set_xlabel('log10(Number of Parameters)')
ax[1].set_ylabel('Test RMSE (knots)')
ax[1].set_title('Pareto Front for Intensity (knots)')

pareto_params_knots = pareto_models_knots['log10_Number of Parameters'].values
pareto_rmse_knots = pareto_models_knots['Test_RMSE_knots'].values

# Draw the Pareto front for RMSE (knots)
for i in range(len(pareto_params_knots)):
    if i > 0:
        ax[1].vlines(x=pareto_params_knots[i], ymin=pareto_rmse_knots[i-1],
                     ymax=pareto_rmse_knots[i],
                     color='k', linewidth=1, zorder=-1)
    ax[1].hlines(y=pareto_rmse_knots[i], xmin=pareto_params_knots[i],
                 xmax=pareto_params_knots[i+1] if i+1 < len(pareto_params_knots) \
                 else max(df['log10_Number of Parameters']),
                 color='k', linewidth=1, zorder=-1)

# Adjust text positions to avoid overlap
adjust_text(texts, ax=ax[1], only_move={'points':'y', 'texts':'y'},
            arrowprops=dict(arrowstyle='-', color='grey'))

plt.tight_layout()
plt.show()
